# Die vergessenen Pandemien: Was die Schweiz aus ihrer Vergangenheit lernen kann
 
Pandemien wie COVID-19 haben uns gezeigt, wie verletzlich moderne Gesellschaften trotz technologischem Fortschritt sein können. Doch die Geschichte kennt viele solcher Gesundheitskrisen – auch in der Schweiz.  
 
Dieses Projekt visualisiert historische Daten zu Pandemien und Sterblichkeit in der Schweiz, um zu zeigen:
- wie Krankheiten sich über Kantone verbreiteten,
- wie Todesraten sich entwickelten,
- und was wir daraus für zukünftige Krisen lernen können.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column, row
from bokeh.palettes import Category10
from bokeh.models import (
    ColumnDataSource, HoverTool, Span, Range1d, LinearAxis, Label, LabelSet,
    PanTool, BoxZoomTool, WheelZoomTool, ResetTool, CrosshairTool,
    NumeralTickFormatter, Legend, Title, Select
)
from bokeh.transform import dodge


In [2]:
data_set1 = pd.read_excel("../Data/1_History_Pandemics.xlsx")
data_set2_mortality = pd.read_excel("../Data/2_All_cantons_1953-1958_Mortality.xlsx")
data_set2_incidence_weekly = pd.read_excel("../Data/2_Data_cantons_incidence_weekly_56_58_NEW.xlsx")
data_set2_population = pd.read_excel("../Data/2_Population_cantons.xlsx")
data_set3 = pd.read_excel("../Data/3_Todesursachen Schweiz ohne Alter 1876-2002.xlsx", header=None)

# Data Profiling with ydata_profiling

In [ ]:
# Data Profiling dataset 1
from ydata_profiling import ProfileReport
profile = ProfileReport(data_set1, 
                        title="OUR DATASET", 
                        sort=None,
                        sensitive=False,
                        explorative=False)

# Create and display the report
# profile.to_notebook_iframe()    # Integrate into a Jupyter notebook
# profile.to_widgets()             # Integrate into a Jupyter notebook, compact
profile.to_file("Dataset1_profiling.html")  # Save the report to a file
# profile

In [ ]:
# Data Profiling
from ydata_set3_profiling import ProfileReport
profile = ProfileReport(data_set2_mortality, 
                        title="OUR DATASET", 
                        sort=None,
                        sensitive=False,
                        explorative=False)

# Create and display the report
# profile.to_notebook_iframe()    # Integrate into a Jupyter notebook
# profile.to_widgets()             # Integrate into a Jupyter notebook, compact
profile.to_file("data_set2_mortality_profiling.html")  # Save the report to a file
# profile

In [ ]:
# Data Profiling
from ydata_profiling import ProfileReport
profile = ProfileReport(data_set2_incidence_weekly, 
                        title="OUR DATASET", 
                        sort=None,
                        sensitive=False,
                        explorative=False)

# Create and display the report
# profile.to_notebook_iframe()    # Integrate into a Jupyter notebook
# profile.to_widgets()             # Integrate into a Jupyter notebook, compact
profile.to_file("data_set2_incidence_weekly_profiling.html")  # Save the report to a file
# profile

In [ ]:
# Data Profiling
from ydata_profiling import ProfileReport
profile = ProfileReport(data_set2_population, 
                        title="OUR DATASET", 
                        sort=None,
                        sensitive=False,
                        explorative=False)

# Create and display the report
# profile.to_notebook_iframe()    # Integrate into a Jupyter notebook
# profile.to_widgets()             # Integrate into a Jupyter notebook, compact
profile.to_file("data_set2_population.html")  # Save the report to a file
# profile

In [ ]:
# Data Profiling
from ydata_profiling import ProfileReport
profile = ProfileReport(data_set3, 
                        title="OUR DATASET", 
                        sort=None,
                        sensitive=False,
                        explorative=False)

# Create and display the report
# profile.to_notebook_iframe()    # Integrate into a Jupyter notebook
# profile.to_widgets()             # Integrate into a Jupyter notebook, compact
profile.to_file("data_set3_population.html")  # Save the report to a file
# profile

---
# Look at the datasets

## Dataset 1

In [ ]:
data_set1.describe()

In [ ]:
data_set1.head(10)

## Dataset 3

In [ ]:
data_set3

In [ ]:
data_set3.head(10)

In [ ]:
header_rows = data_set3.iloc[3:6].fillna(method='ffill', axis=1)
combined_headers = header_rows.apply(lambda x: ' | '.join(x.dropna().astype(str)), axis=0)

# Effektive Daten ab Zeile 10 (Index 9)
data = data_set3.iloc[9:].copy()
data.columns = combined_headers
data = data.reset_index(drop=True)

for col in data.columns:
    print(col)

---
## Dataset 1. Pandemic Death Rates (Bar Chart)


In 2020, the COVID-19 pandemic disrupted life across the globe. In Switzerland, as in many countries, society paused, reeled, and eventually adapted. As the emergency fades, a question lingers:

> **Have we really learned from this crisis—or are we doomed to forget, again?**

This project invites you on a journey. A journey through 130 years of Swiss pandemic history, told through data: deaths, diseases, and resilience. We visualize key insights from historical records to better understand how pandemics shaped our past—and how they can guide our future.


### How Deadly Were Past Pandemics?
When people think of pandemics, COVID-19 is top of mind. But how does it compare to earlier pandemics?

To answer this, we looked at death rates from major pandemics in Switzerland from 1889 to 2020, measured per 100,000 people. The bar chart below reveals the toll of each crisis:



In [ ]:
pandemic_years = [1889, 1918, 1957, 1968, 2009, 2020]  # Major pandemic years
pandemic_data = data_set1[data_set1['Jahr'].isin(pandemic_years)].copy()

# Erstelle eine neue Spalte für die Todesfälle pro Pandemie
pandemic_data['Todesfälle_100000'] = pandemic_data['Todesfälle_Grippe_100000'].fillna(0)
# Ersetze COVID-Werte für 2020
pandemic_data.loc[pandemic_data['Jahr'] == 2020, 'Todesfälle_100000'] = pandemic_data.loc[pandemic_data['Jahr'] == 2020, 'Todesfälle_Covid_100000'].fillna(0)

# Create the figure and axis
fig, ax = plt.subplots(figsize=(12, 7))

# Set width of bars
bar_width = 0.6
index = np.arange(len(pandemic_years))

# Erstelle eine Farbliste (alle blau außer COVID in orange)
colors = ['#1f77b4', '#1f77b4', '#1f77b4', '#1f77b4', '#1f77b4', '#ff7f0e']

# Create the bars (nur ein Balken pro Pandemie)
bars = ax.bar(index, pandemic_data['Todesfälle_100000'], 
              bar_width, color=colors, alpha=0.8)

# Add labels, title
ax.set_ylabel('Deaths per 100,000 Population', fontsize=12)
ax.set_title('Comparison of Major Pandemic Death Rates', fontsize=20)

# Add pandemic names with years directly in the x-tick labels
pandemic_names_with_years = ["Russian Flu (1889)", "Spanish Flu (1918)", "Asian Flu (1957)", 
                           "Hong Kong Flu (1968)", "Swine Flu (2009)", "COVID-19 (2020)"]
ax.set_xticks(index)
ax.set_xticklabels(pandemic_names_with_years, rotation=45, ha='right')

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    if height > 0:
        ax.annotate(f'{height:.1f}', 
                   xy=(bar.get_x() + bar.get_width() / 2, height),
                   xytext=(0, 3), textcoords="offset points", 
                   ha='center', va='bottom')

# Füge eine Legende hinzu
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#1f77b4', label='Historical Pandemics'),
    Patch(facecolor='#ff7f0e', label='COVID-19')
]
ax.legend(handles=legend_elements, loc='upper right')

# Adjust layout
plt.tight_layout()
plt.show()

### Key Findings:

- The 1918 Spanish Flu had by far the highest death rate in Swiss history—more than five times higher than COVID-19.
- Pandemics in 1957 (Asian Flu) and 1968 (Hong Kong Flu) caused significant but lesser mortality.
- The 2009 Swine Flu was comparatively mild in Switzerland.

>COVID-19 was not the deadliest pandemic in Swiss history. In fact, the Spanish Flu of 1918 remains unmatched in scale. But history shows: even ‘moderate’ pandemics can leave lasting scars.


---
## Dataset 1: Population and Pandemic Deaths Over Time

### Time, Population, and Mortality: The Bigger Picture
Pandemic impact doesn’t occur in isolation—it happens in the context of a growing society. To understand the broader picture, we plotted pandemic death rates alongside population growth over more than a century.

**Tip:** Hover over the lines in the chart to explore each year. You’ll see how many people lived in Switzerland, and how many died from influenza or COVID-19 during that time.

In [73]:
output_notebook()


pandemic_years = [1889, 1918, 1957, 1968, 2009, 2020]
pandemic_names = ["Russian Flu", "Spanish Flu", "Asian Flu", "Hong Kong Flu", "Swine Flu", "COVID-19"]

# Daten vorbereiten
pandemic_data = data_set1.copy()
highlight_data = data_set1[data_set1['Jahr'].isin(pandemic_years)].copy()


source_main = ColumnDataSource(pandemic_data)
source_highlights = ColumnDataSource(highlight_data)


p = figure(
    title="Pandemic Mortality vs. Population Growth (1889-2020)",
    height=600,
    width=950,
    x_axis_label="Year",
    y_axis_label="Population",
    tools=""  
)


p.yaxis.formatter = NumeralTickFormatter(format="0,0")
p.y_range = Range1d(0, pandemic_data['Population'].max() * 1.1)


max_deaths = max(pandemic_data['Todesfälle_Grippe_100000'].max(), 
               pandemic_data['Todesfälle_Covid_100000'].max())
p.extra_y_ranges = {"deaths": Range1d(0, max_deaths * 1.1)}
p.add_layout(LinearAxis(y_range_name="deaths", axis_label="Deaths per 100,000 Population"), 'right')


population_line = p.line(
    x='Jahr', y='Population', source=source_main,
    line_width=3, line_dash='dashed', color='black',
    alpha=0.7, legend_label="Population"
)


flu_line = p.line(
    x='Jahr', y='Todesfälle_Grippe_100000', source=source_main,
    y_range_name='deaths', line_width=3, color=Category10[3][0],
    alpha=0.8, legend_label="Influenza Deaths"
)


covid_line = p.line(
    x='Jahr', y='Todesfälle_Covid_100000', source=source_main,
    y_range_name='deaths', line_width=3, color=Category10[3][1],
    alpha=0.8, legend_label="COVID-19 Deaths"
)


for year in pandemic_years:
    if year in pandemic_data['Jahr'].values:
        vline = Span(location=year, dimension='height', 
                   line_color='darkgray', line_dash='dashed', line_width=1)
        p.add_layout(vline)


flu_circles = p.circle(
    x='Jahr', y='Todesfälle_Grippe_100000', source=source_highlights,
    y_range_name='deaths', size=10, color=Category10[3][0],
    alpha=0.8, line_color='black', line_width=1
)


covid_circles = p.circle(
    x='Jahr', y='Todesfälle_Covid_100000', source=source_highlights,
    y_range_name='deaths', size=10, color=Category10[3][1],
    alpha=0.8, line_color='black', line_width=1
)



# Tooltip für Crosshair
hover = HoverTool(
    tooltips=[
        ("Year", "@Jahr"),
        ("Population", "@Population{0,0}"),
        ("Influenza Deaths", "@{Todesfälle_Grippe_100000}{0.0} per 100k"),
        ("COVID Deaths", "@{Todesfälle_Covid_100000}{0.0} per 100k")
    ],
    renderers=[population_line],
    mode='vline',  
    line_policy='nearest'
)

# Create a properly defined CrosshairTool
crosshair = CrosshairTool(line_color="gray", line_alpha=0.6)

# In 

tools = [
    PanTool(),
    BoxZoomTool(),
    WheelZoomTool(),
    ResetTool(),
    crosshair,
    hover,
]
p.add_tools(*tools)

# Legende und Styling
p.legend.location = "top_left"
p.legend.click_policy = "hide"
p.legend.background_fill_alpha = 0.7

p.title.text_font_size = '14pt'
p.title.align = 'center'

p.grid.grid_line_alpha = 0.3

# Plot anzeigen
show(p)

Loading BokehJS ...

>“A growing population does not automatically mean higher mortality—if health systems adapt. Still, sharp spikes in 1918 and 2020 show that even modern nations remain vulnerable when overwhelmed.”

### Key Findings:
- Switzerland's population grew from under 3 million in 1880 to over 8 million by 2022.
- Despite this growth, pandemic death rates spiked dramatically only in select years—especially in 1918 and 2020.
- Medical and public health advances appear to have helped reduce death rates in later pandemics.

---
## Dataset 1: Excess Mortality Over Time



### Excess Mortality Over Time
To understand the true cost of pandemics, we looked beyond reported causes of death. Sometimes, people die because of a pandemic, but not from the disease itself—indirect effects like delayed treatments, overwhelmed hospitals, or social disruptions can all lead to excess deaths.

This is where excess mortality becomes essential. It measures how many people died above or below what we would statistically expect in a normal year, based on historical trends.

**How to read the graph:**

- Each dot represents a year between 1880 and 2022.
- Red dots mean more people died than expected → positive excess mortality.

    This often occurs during severe flu seasons, pandemics, heatwaves, or crises.

- Green dots mean fewer people died than expected → negative excess mortality.

    This can reflect milder flu seasons, improved healthcare, or social measures like lockdowns reducing accidents.
    
- The gray line shows the trend over time.
- Vertical lines mark known pandemic years like 1918, 1957, and 2020.
- The horizontal dashed line at 0% represents the baseline: deaths were as expected that year.

In [74]:
output_notebook()

# Prepare the data
df = data_set1.copy()

# Drop rows where either 'Jahr' or 'Überasterblichkeit_Alles' is missing
df = df.dropna(subset=['Jahr', 'Überasterblichkeit_Alles'])

# Create a color column based on excess mortality
df['color'] = df['Überasterblichkeit_Alles'].apply(lambda x: 'red' if x > 0 else 'green')

# Create the ColumnDataSource
source = ColumnDataSource(df)

# Create the figure
p = figure(
    title="Excess Mortality in Switzerland (1880–2022)",
    height=600,
    width=950,
    x_axis_label="Year",
    y_axis_label="Excess Mortality (%)",
    tools="pan,box_zoom,reset,wheel_zoom,hover",
    tooltips=[
        ("Year", "@Jahr"),
        ("Excess Mortality", "@Überasterblichkeit_Alles{0.0}%")
    ]
)

# Line showing the trend
p.line('Jahr', 'Überasterblichkeit_Alles', source=source, line_width=3, color="gray", alpha=0.6)

# Colored points
p.scatter(
    'Jahr', 'Überasterblichkeit_Alles', source=source,
    size=8, color='color', line_color='black', line_width=1
)

# Horizontal line at 0%
zero_line = Span(location=0, dimension='width', line_color='black', line_dash='dashed', line_width=2)
p.add_layout(zero_line)

# Mark pandemic years
pandemic_years = [1889, 1918, 1957, 1968, 2009, 2020]
for year in pandemic_years:
    if year in df['Jahr'].values:
        vline = Span(location=year, dimension='height', line_color='darkgray', line_dash='dotted', line_width=1)
        p.add_layout(vline)

# Styling
p.title.text_font_size = '14pt'
p.title.align = 'center'
p.y_range = Range1d(-15, 50)

p.grid.grid_line_alpha = 0.3

# Show the plot
show(p)


Loading BokehJS ...

## Dataset 2 approx 7

In [75]:
# Population of the individual cantons and Switzerland as a whole
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import Category20_20
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource

# Bokeh im Notebook aktivieren
output_notebook()

# Daten einlesen
population_df = pd.read_excel("../Data/2_Population_cantons.xlsx")

# Erste Spalte korrekt als "Year" benennen
population_df.rename(columns={population_df.columns[0]: "Year"}, inplace=True)
population_df = population_df.drop(columns='CH')

# Zahlen bereinigen und in float umwandeln (z. B. '1’000’000' → 1000000.0)
for col in population_df.columns[1:]:
    population_df[col] = (
        population_df[col]
        .astype(str)
        .str.replace("'", "")
        .str.replace(",", "")
        .astype(float)
    )

# Daten ins Long-Format bringen
df_long = population_df.melt(id_vars="Year", var_name="Canton", value_name="Population")
# Werte durch 1 Mio teilen:
df_long["Population"] = df_long["Population"] / 1_000_000

# Farbpalette für max. 20 Farben
palette = Category20_20

# Hauptplot vorbereiten
p = figure(title="Bevölkerungsentwicklung der Schweizer Kantone (1950–1970)",
           x_axis_label="Jahr", y_axis_label="Einwohnerzahl (in Mio.)",
           width=950, height=550, tools="pan,wheel_zoom,box_zoom,reset,hover,save")

# Kantone alphabetisch sortieren
cantons = sorted(df_long["Canton"].unique())

# Linien für alle Kantone zeichnen
for i, canton in enumerate(cantons):
    df_k = df_long[df_long["Canton"] == canton]
    source = ColumnDataSource(df_k)
    p.line(x='Year', y='Population', source=source,
           line_width=2, legend_label=canton,
           color=palette[i % len(palette)])  # Wiederholt Palette bei >20 Kantonen

# Hover aktivieren
hover = p.select_one(HoverTool)
hover.tooltips = [("Kanton", "@Canton"), ("Jahr", "@Year"), ("Einwohner (Mio.)", "@Population{0.00}")]

# Interaktive Legende
p.legend.location = "right"
p.legend.click_policy = "hide"

p.yaxis.formatter = NumeralTickFormatter(format= '0.00')
# Plot anzeigen
show(p)

Loading BokehJS ...

In [76]:
import panel as pn
# Bokeh in Notebook aktivieren
output_notebook()

# Daten einlesen
incidence_df = pd.read_excel("../Data/2_Data_cantons_incidence_weekly_56_58_NEW.xlsx")

incidence_monthly = incidence_df.copy()

# Drop unnecessary columns
incidence_monthly = incidence_monthly.drop(columns='Parameter')

# Create YearMonth and group
incidence_monthly['YearMonth'] = incidence_monthly['StartReportingPeriod'].dt.to_period('M')
incidence_monthly = incidence_monthly.drop(columns=['StartReportingPeriod', 'EndReportingPeriod', 'Month'])
incidence_monthly = incidence_monthly.groupby('YearMonth').sum().reset_index()

# Convert YearMonth to a real datetime (first of each month)
incidence_monthly['Date'] = incidence_monthly['YearMonth'].dt.to_timestamp()

# Create source
source_incidence_monthly = ColumnDataSource(incidence_monthly)

# Plot
p = figure(title="Monthly Influenza Cases (1953–1958)",
           x_axis_type="datetime",
           width=950, height=550,
           x_axis_label="Date", y_axis_label="Number of Influenza cases",
           tools="pan,wheel_zoom,box_zoom,reset,hover,save")

# Line chart
p.line(x='Date', y='CH', source=source_incidence_monthly,  # <-- 'CH' column for total cases
       line_width=3, color=Category10[3][2], legend_label="Influenza cases")

# Hover-Tool
hover = p.select_one(HoverTool)
hover.tooltips = [
    ("Date", "@Date{%Y-%m}"),
    ("Influenza Cases", "@CH")
]
hover.formatters = {
    '@Date': 'datetime'
}
hover.mode = 'vline'

# Legend
p.legend.location = "top_left"
p.legend.click_policy = "hide"

# Plot anzeigen
show(p)
# Dropdown-Widget
canton_select = pn.widgets.Select(name='Select Canton', options=cantons, value=cantons[0])


Loading BokehJS ...

In [77]:


# Bokeh im Notebook aktivieren
output_notebook()

# Daten einlesen
incidence_df = pd.read_excel("../Data/2_Data_cantons_incidence_weekly_56_58_NEW.xlsx")

# Daten vorbereiten
incidence_weekly = incidence_df.copy()

# Woche erstellen und unnötige Spalten löschen
incidence_weekly['weekly'] = incidence_weekly['StartReportingPeriod'].dt.to_period('W')
incidence_weekly = incidence_weekly.drop(columns=['StartReportingPeriod', 'EndReportingPeriod', 'Month'])

# Nach Woche gruppieren und Summieren
incidence_weekly = incidence_weekly.groupby('weekly').sum().reset_index()

# Wochenstartdatum extrahieren
incidence_weekly['Date'] = incidence_weekly['weekly'].dt.start_time

# Datenquelle für Bokeh
source = ColumnDataSource(incidence_weekly)

# Plot erstellen
p = figure(
    title="Weekly Influenza Cases (1952–1958)",
    x_axis_type="datetime",
    width=950,
    height=550,
    x_axis_label="Date",
    y_axis_label="Number of Influenza Cases",
    tools="pan,wheel_zoom,box_zoom,reset,hover,save"
)

# Linie zeichnen
p.line(
    x='Date', 
    y='CH',  # Spalte für gesamte Schweiz
    source=source,
    line_width=3,
    color=Category10[3][2],
    legend_label="Switzerland (CH)"
)

# Hover-Tool anpassen
hover = p.select_one(HoverTool)
hover.tooltips = [
    ("Date", "@Date{%F}"),
    ("Cases", "@CH")
]
hover.formatters = {'@Date': 'datetime'}
hover.mode = 'vline'

# Legende anpassen
p.legend.location = "top_left"
p.legend.click_policy = "hide"

# Plot anzeigen
show(p)

Loading BokehJS ...

## The Invisible Peaks: Influenza Mortality in the Shadow of Total Deaths (1953–1958)
While influenza rarely dominates the headlines of mortality statistics, its seasonal impact is undeniable when viewed alongside the total number of deaths. Between 1953 and 1958 in Switzerland, influenza outbreaks show up as sharp, low peaks — especially in early 1953, 1956, and most notably in late 1957.

These spikes correspond to known influenza epidemics — most dramatically the Asian Flu (H2N2), which began spreading globally in mid-1957 and reached Europe by the end of that year. In Switzerland, this wave caused a visible rise in influenza-specific deaths, peaking in December 1957–January 1958 [source: World Health Organization].

Despite this, influenza-related mortality appears modest compared to the total number of deaths per month — which remain relatively stable and are shaped by broader trends such as aging populations, chronic disease, and seasonal factors like cold winters increasing respiratory-related mortality in general.

Still, influenza’s seasonal visibility in this dataset reminds us of a critical lesson: even when not catastrophic, flu seasons burden the healthcare system and add to existing mortality pressures. The hidden weight of influenza may not always be dramatic in scale, but its recurring role is both epidemiologically and socially significant.

- Flu peaks are visible in winters of 1953, 1956, and especially late 1957.

- The 1957/58 spike aligns with the global H2N2 pandemic (Asian Flu).

- Total deaths remain stable but include many causes — including chronic conditions and general respiratory illnesses.

- Contextual awareness (e.g., population aging or cold spells) is essential to interpret mortality patterns meaningfully.


In [78]:
# # Comparison of influenza deaths and general deaths (per canton and Switzerland) (bis August 1958, da Daten fehlen)
output_notebook()

mortality_df = pd.read_excel("../Data/2_All_cantons_1953-1958_Mortality.xlsx")

# Parameter-Spalte normalisieren
mortality_df["Parameter"] = mortality_df["Parameter"].str.strip().str.lower()
mortality_df["Parameter"] = mortality_df["Parameter"].replace({
    "deaths total": "deaths total",
    "total deaths": "deaths total",
    "total death": "deaths total"
})

# Monatsnamen in Zahlen umwandeln
month_map = {
    'january': 1, 'february': 2, 'march': 3, 'april': 4,
    'may': 5, 'june': 6, 'july': 7, 'august': 8,
    'september': 9, 'october': 10, 'november': 11, 'december': 12
}
mortality_df["Month"] = mortality_df["Month"].str.strip().str.lower().map(month_map)

# Influenza- und Total-Deaths für CH extrahieren
influenza_ch = mortality_df[mortality_df["Parameter"] == "deaths influenza"][["Year", "Month", "CH"]]
total_ch = mortality_df[mortality_df["Parameter"] == "deaths total"][["Year", "Month", "CH"]]

# Jahres- und Monatsweise Zusammenführen
comparison_df = influenza_ch.copy()
comparison_df = comparison_df.rename(columns={"CH": "Influenza_Deaths"})
comparison_df["Total_Deaths"] = total_ch["CH"].values

# Datum zusammenbauen
comparison_df["Date"] = pd.to_datetime(dict(year=comparison_df["Year"], month=comparison_df["Month"], day=1))

# Nur bis August 1958 behalten
comparison_df = comparison_df[(comparison_df["Year"] < 1958) | ((comparison_df["Year"] == 1958) & (comparison_df["Month"] <= 8))]


source = ColumnDataSource(comparison_df)

colors = Category10[3]

# Plot erstellen
p = figure(title="Comparison: Influenza Deaths vs. Total Deaths in Switzerland (1953–1958)",
           x_axis_type="datetime",
           width=950, height=550,
           x_axis_label="Date", y_axis_label="Anzahl Todesfälle",
           tools="pan,wheel_zoom,box_zoom,reset,hover,save")

# Linie für Influenza-Todesfälle
p.line(x='Date', y='Influenza_Deaths', source=source,
       line_width=3, color=colors[0], legend_label="Influenza Deaths")

# Linie für Gesamttodesfälle
p.line(x='Date', y='Total_Deaths', source=source,
       line_width=3, color=colors[1], legend_label="Total Deaths")

# Hover-Tool 
hover = p.select_one(HoverTool)
hover.tooltips = [
    ("Datum", "@Date{%Y-%m}"),
    ("Influenza-Todesfälle", "@Influenza_Deaths{0,0}"),
    ("Gesamte Todesfälle", "@Total_Deaths{0,0}")
]
hover.formatters = {'@Date': 'datetime'}
hover.mode = 'vline'

# Interaktive Legende
p.legend.location = "top_left"
p.legend.click_policy = "hide"

# Plot anzeigen
show(p)


Loading BokehJS ...

Hinweis: Für die Monate September bis Dezember 1958 liegen keine vollständigen Total-Todesfalldaten vor. Die Analysen basieren daher auf den vollständig dokumentierten Jahren 1953 bis August 1958.

In [79]:
print(comparison_df)

    Year  Month  Influenza_Deaths  Total_Deaths       Date
0   1953      1              42.0        4573.0 1953-01-01
1   1953      2             475.0        5174.0 1953-02-01
2   1953      3             446.0        5250.0 1953-03-01
3   1953      4              78.0        4193.0 1953-04-01
4   1953      5              17.0        4090.0 1953-05-01
..   ...    ...               ...           ...        ...
63  1958      4              38.0        4187.0 1958-04-01
64  1958      5              17.0        4067.0 1958-05-01
65  1958      6               9.0        3812.0 1958-06-01
66  1958      7               6.0        3853.0 1958-07-01
67  1958      8               4.0        3499.0 1958-08-01

[68 rows x 5 columns]


(Plot nochmals prüfen, stimmen die daten so wirklich?? und sources)
Around October–November 1957, over 30,000 weekly influenza cases were reported. Yet, monthly deaths peaked around 300 (which is still a significant number, but small relative to infections).

Why so many cases, but "few" deaths?

- High transmissibility, low fatality: The 1957 H2N2 flu virus was highly contagious but had a case fatality rate of around 0.1–0.3%. That means: for every 1,000 people infected, only 1–3 died on average.

- Younger populations were less affected: The virus disproportionately affected the elderly. In Switzerland at the time, the general population was younger — which may have reduced overall mortality.

- Better medical care & antibiotics: Antibiotics were increasingly available to treat secondary bacterial infections, like pneumonia, which often kill flu patients. Hospitals were better prepared compared to 1918.

- Vaccination campaigns had already started: In some countries, early vaccination efforts helped mitigate mortality — even if they didn’t prevent all infections.

CDC: Pandemic Influenza Past https://www.cdc.gov/pandemic-flu/index.html, WHO archive: 1957–58 pandemic overview,Swiss health data (historical context)

In [ ]:
output_notebook()

# Load both datasets
cases_df = pd.read_excel("../Data/2_Data_cantons_incidence_weekly_56_58_NEW.xlsx")
deaths_df = pd.read_excel("../Data/2_All_cantons_1953-1958_Mortality.xlsx")

# Clean and prepare weekly case data
cases_df = cases_df[cases_df["Parameter"] == "Cases Influenza"]
weekly_ch = cases_df[["StartReportingPeriod", "CH"]].rename(columns={"StartReportingPeriod": "Date", "CH": "Weekly_Cases"})
weekly_ch["Date"] = pd.to_datetime(weekly_ch["Date"])

# Clean and prepare monthly death data
deaths_df["Parameter"] = deaths_df["Parameter"].str.strip().str.lower()
deaths_df = deaths_df[deaths_df["Parameter"] == "deaths influenza"]

month_map = {
    'january': 1, 'february': 2, 'march': 3, 'april': 4,
    'may': 5, 'june': 6, 'july': 7, 'august': 8,
    'september': 9, 'october': 10, 'november': 11, 'december': 12
}
deaths_df["Month"] = deaths_df["Month"].str.lower().map(month_map)
deaths_df["Date"] = pd.to_datetime(dict(year=deaths_df["Year"], month=deaths_df["Month"], day=1))
monthly_ch = deaths_df[["Date", "CH"]].rename(columns={"CH": "Monthly_Deaths"})

# Filter for 1957 and 1958 only
weekly_ch = weekly_ch[(weekly_ch["Date"].dt.year >= 1957) & (weekly_ch["Date"].dt.year <= 1958)]
monthly_ch = monthly_ch[(monthly_ch["Date"].dt.year >= 1957) & (monthly_ch["Date"].dt.year <= 1958)]

# Plot
p = figure(title="Weekly Influenza Cases and Monthly Deaths (1957–1958)",
           x_axis_type="datetime",
           width=950, height=550,
           x_axis_label="Date", y_axis_label="Count",
           tools="pan,wheel_zoom,box_zoom,reset,hover,save")

p.line(x='Date', y='Weekly_Cases', source=ColumnDataSource(weekly_ch),
       line_width=2, color=Category10[3][0], legend_label="Weekly Cases")

p.circle(x='Date', y='Monthly_Deaths', source=ColumnDataSource(monthly_ch),
         size=8, color=Category10[3][1], legend_label="Monthly Influenza Deaths")

hover = p.select_one(HoverTool)
hover.tooltips = [("Date", "@Date{%F}"), ("Count", "$y{0,0}")]
hover.formatters = {'@Date': 'datetime'}

p.legend.location = "top_left"
p.legend.click_policy = "hide"

show(p)

Loading BokehJS ...

## Dataset 3

### Long-Term Shifts in Causes of Death in Switzerland (1876–2002)
As we return from our data journey, we zoom out to see the bigger picture. The history of pandemics is only one thread in a much broader transformation:
How the causes of death in Switzerland have changed over nearly 150 years.

**The Decline of Infectious Diseases**
In the late 19th and early 20th centuries, infectious diseases such as tuberculosis, measles, diphtheria, scarlet fever, and whooping cough were leading causes of death in Switzerland.

- Several key developments contributed to their decline:

- Public sanitation and clean water infrastructure drastically reduced the spread of waterborne and respiratory diseases.

- Widespread vaccination campaigns targeted diseases like smallpox, measles, and diphtheria, dramatically lowering incidence and mortality.

- The discovery and use of antibiotics, starting in the 1940s, enabled effective treatment of bacterial infections that had once been fatal.

In the 19th century, tuberculosis alone accounted for a major share of deaths in cities like Bern.

*Source: PMC article on tuberculosis mortality in Bern*

In [ ]:
output_notebook()

# Vorbereitung
df = data.copy()
df = df.loc[:, ~df.columns.duplicated()]

# Spaltenauswahl: Jahr + Krankheiten
df['Year'] = pd.to_numeric(df['Jahr'], errors='coerce')

# Wähle Krankheitsgruppen
cols = [
    "Infektions- und parasitäre Krankheiten | Pocken, Scharlach, Masern, Typhus, Diphtherie, Keuchhusten | Total",
    "Atmungsorgane | Total | Aids",  # Diese musst du ggf. anpassen
    "Neubildungen | Total | chitis"  # Diese auch ggf. anpassen
]

# Umbenennen für bessere Lesbarkeit
col_map = {
    cols[0]: "Infectious Diseases",
    cols[1]: "Respiratory Diseases",
    cols[2]: "Neoplasms"
}

df_subset = df[["Year"] + cols].dropna()
for col in cols:
    df_subset[col] = pd.to_numeric(df_subset[col], errors='coerce')
df_subset[cols] = df_subset[cols].clip(lower=0)

# Datenquelle
source = ColumnDataSource(df_subset)

# Figure
p = figure(title="Major Causes of Death in Switzerland (Tooltip only on Respiratory Diseases)",
           x_axis_label="Year", y_axis_label="Deaths",
           width=950, height=550, tools="")

# Linien zeichnen & speichern
colors = Category10[len(cols)]
renderers = {}

for i, col in enumerate(cols):
    readable = col_map[col]
    line = p.line(x='Year', y=col, source=source,
                  line_width=3 if readable == "Respiratory Diseases" else 2,
                  color=colors[i],
                  alpha=0.9 if readable == "Respiratory Diseases" else 0.6,
                  legend_label=readable)
    renderers[readable] = line

hover = HoverTool(
    tooltips=[
        ("Year", "@Year"),
        ("Respiratory Deaths", f"@{{{cols[1]}}}{{0,0}}"),
        ("Infectious Deaths", f"@{{{cols[0]}}}{{0,0}}"),
        ("Neoplasms Deaths", f"@{{{cols[2]}}}{{0,0}}")
    ],
    renderers=[renderers["Respiratory Diseases"]],
    mode='vline'
)

p.add_tools(hover)

# Styling
p.legend.location = "top_left"
p.legend.click_policy = "hide"

show(p)


Loading BokehJS ...

### A Direct Comparison Across Time
This visual offers a unique opportunity:
It lets you freely compare any two years in Swiss mortality history—side by side.

In the example shown (1876 vs. 2004), the differences are striking:

- Infectious diseases, once dominant, had almost disappeared by 2004.

- Meanwhile, cancers and respiratory diseases had become leading causes of death.

    The chart is interactive: simply select two years, and watch how the causes shift.
    It invites you to explore your own questions—whether you're interested in the long-term decline of epidemics, the rise of chronic illnesses, or the effects of public health interventions.

This comparison isn’t just about numbers.
It makes visible how our medical history, environment, and behaviors have fundamentally changed what it means to get sick—and what it means to die.



In [82]:
output_notebook()

# --- Prepare data (use your cleaned DataFrame) ---
df = data_subset.copy()
df['Year'] = df['Year'].dt.year  # Convert to int for easier selection

# Extract available years and causes
years = sorted(df['Year'].unique())
causes = [col for col in df.columns if col != 'Year']

# Initial selection
year_a = years[0]
year_b = years[-1]

def get_comparison_data(year1, year2):
    row1 = df[df['Year'] == year1][causes].iloc[0]
    row2 = df[df['Year'] == year2][causes].iloc[0]
    return pd.DataFrame({
        'Cause': causes,
        'Year A': row1.values,
        'Year B': row2.values
    })

comparison_df = get_comparison_data(year_a, year_b)
source = ColumnDataSource(comparison_df)

# --- Create bar plot ---
p = figure(y_range=comparison_df['Cause'], height=400, width=800,
           title=f"Causes of Death: {year_a} vs {year_b}",
           x_axis_label="Number of Deaths", toolbar_location=None)

p.hbar(y=dodge('Cause', -0.2, range=p.y_range), right='Year A', height=0.35,
       source=source, color=Category10[3][0], legend_label=str(year_a))

p.hbar(y=dodge('Cause',  0.2, range=p.y_range), right='Year B', height=0.35,
       source=source, color=Category10[3][1], legend_label=str(year_b))

p.ygrid.grid_line_color = None
p.legend.location = "top_right"
p.legend.orientation = "horizontal"

# --- Dropdown widgets ---
select_a = Select(title="Select Year", value=str(year_a), options=[str(y) for y in years])
select_b = Select(title="Select Year", value=str(year_b), options=[str(y) for y in years])

def update(attr, old, new):
    ya = int(select_a.value)
    yb = int(select_b.value)
    updated_df = get_comparison_data(ya, yb)
    source.data = ColumnDataSource.from_df(updated_df)
    p.title.text = f"Causes of Death: {ya} vs {yb}"
    p.legend.items[0].label = str(ya)
    p.legend.items[1].label = str(yb)

select_a.on_change("value", update)
select_b.on_change("value", update)

# --- Show layout ---
show(column(row(select_a, select_b), p))


Loading BokehJS ...

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/js_callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



### Breaking Down Epidemics: The Disappearance of Specific Infectious Diseases

To go even deeper into the story of public health progress, we analyzed the **subgroups of infectious diseases** individually.
Instead of treating infectious deaths as a single category, this visualization breaks them down into their historical components:

* **Smallpox**
* **Measles**
* **Scarlet fever**
* **Diphtheria**
* **Typhus**
* **Whooping cough**

These diseases once claimed thousands of lives every year—especially among children. In the late 1800s and early 1900s, they were among the most feared causes of death in Switzerland.

But over time, something remarkable happened.

* One after another, these lines drop toward zero.
* By the end of the 20th century, most of these diseases had effectively disappeared from the mortality statistics.

> The thick gray line at the top of the chart shows the total deaths from infectious diseases.
> The colored lines underneath it represent each subgroup—declining at different speeds.

This chart does more than show death counts.
It visualizes the impact of **vaccines**, **antibiotics**, **public health systems**, and **collective behavior**.
It is, in a sense, a portrait of one of modern medicine’s greatest achievements:

---

**Sources on the Decline of Specific Infectious Diseases in Switzerland**

 1. **Historical Disease Burden in Switzerland**

* **Swiss Federal Statistical Office (BFS)**
  Die offiziellen Todesursachenstatistiken zeigen die rückläufige Entwicklung von Krankheiten wie Masern, Keuchhusten, Diphtherie etc. ab dem 20. Jahrhundert.
  ➤ [BFS – Causes of Death Statistics](https://www.bfs.admin.ch/bfs/en/home/statistics/health/state-of-health/mortality-causes-death.html)

2. **The Role of Vaccination**

* **European Centre for Disease Prevention and Control (ECDC)** – Impfprogramme in Europa haben Krankheiten wie Diphtherie, Keuchhusten und Masern stark reduziert.
  ➤ [ECDC – Vaccine-preventable diseases](https://www.ecdc.europa.eu/en/immunisation-vaccine-preventable-diseases)

* **World Health Organization (WHO)** – Success stories of vaccination in Europe, incl. measles, diphtheria, smallpox.
  ➤ [WHO – Immunization in the European Region](https://www.who.int/europe/health-topics/vaccines-and-immunization)

3. **Disease Elimination in Switzerland**

* **Smallpox** was officially eradicated globally in 1980, but Switzerland had already stopped routine vaccination in 1972.
  ➤ [WHO Smallpox Eradication Timeline](https://www.who.int/news-room/fact-sheets/detail/smallpox)

* **Diphtheria, Scarlet fever, Whooping cough (pertussis)**: sharp declines after vaccine introduction in mid-20th century.

> For example, in 1945, whooping cough caused over 1,000 deaths in Switzerland. Today, the number is close to zero (source: BFS).

4. **Academic Literature**

* **Staub K, Rühli FJ, Woitek U, Pfister C.**
  *Historical mortality data for Switzerland 1876–2015.*
  This paper provides clean historical cause-of-death data and is frequently cited.
  ➤ [PMID: 30318199](https://pubmed.ncbi.nlm.nih.gov/30318199/)

* **Global Burden of Disease Project** (Institute for Health Metrics and Evaluation):
  Offers data visualizations showing long-term disease shifts.
  ➤ [GBD Data Explorer](https://vizhub.healthdata.org/gbd-results/)




In [117]:
output_notebook()

# Clean base dataframe
df = data.copy()
df = df.loc[:, ~df.columns.duplicated()]

infectious_cols = [
    col for col in df.columns
    if col.startswith("Infektions-") and
    any(kw in col.lower() for kw in ["pocken", "masern", "schar", "typhus", "diph", "keuch"]) and
    not col.strip().endswith("Total")  # ← schließt die vorhandene Total-Spalte aus!
]


df['Year'] = pd.to_numeric(df['Jahr'], errors='coerce')

# Subset and clean
df_subset = df[['Year'] + infectious_cols].dropna()
for col in infectious_cols:
    df_subset[col] = pd.to_numeric(df_subset[col], errors='coerce')
df_subset[infectious_cols] = df_subset[infectious_cols].clip(lower=0)

# ➕ Calculate new total column
df_subset['Total_Infectious'] = df_subset[infectious_cols].sum(axis=1)

# Prepare for Bokeh
source = ColumnDataSource(df_subset)
colors = Category10[max(3, len(infectious_cols))]

# Create figure
p = figure(title="Infectious Disease Subgroups in Switzerland (1876–2002)",
           x_axis_label="Year", y_axis_label="Deaths",
           width=950, height=550, tools="")

# Draw all subgroup lines
for i, col in enumerate(infectious_cols):
    label = col.split('|')[-1].strip()
    label = label.replace('Schar-', 'Scarlet fever') \
                 .replace('Pocken', 'Smallpox') \
                 .replace('Masern', 'Measles') \
                 .replace('Typhus,', 'Typhus') \
                 .replace('Diph-', 'Diphtheria') \
                 .replace('Keuch-', 'Whooping cough')
    
    p.line(x='Year', y=col, source=source,
           line_width=2, color=colors[i % len(colors)],
           legend_label=label)

# Draw new calculated total line
total_line = p.line(x='Year', y='Total_Infectious', source=source,
                    line_width=3, color='darkgray', alpha=0.8,
                    legend_label='Total')

# Tooltip on total line only
tooltip_items = [("Year", "@Year")]
label_map = {
    "Schar-": "Scarlet fever",
    "Pocken": "Smallpox",
    "Masern": "Measles",
    "Typhus,": "Typhus",
    "Diph-": "Diphtheria",
    "Keuch-": "Whooping cough"
}
for col in infectious_cols:
    label = col.split('|')[-1].strip()
    for short, full in label_map.items():
        label = label.replace(short, full)
    tooltip_items.append((label, f"@{{{col}}}"))

hover = HoverTool(
    tooltips=tooltip_items,
    mode='vline',
    renderers=[total_line]  # Only when hovering over the total line
)
p.add_tools(hover)

# Styling
p.legend.location = "top_right"
p.legend.click_policy = "hide"

show(p)


Loading BokehJS ...

### **Conclusion: Remembering, Understanding, Preparing**

Our journey through more than 140 years of health data has revealed one clear truth:
Pandemics have never been rare exceptions—they are a recurring part of history.

We explored how diseases like influenza spread across cantons, how mortality evolved, and how the causes of death shifted dramatically over time.
What was once dominated by smallpox, measles, or diphtheria is now shaped by cancer and chronic illness.

But even the 21st century remains vulnerable. COVID-19 was not a one-time shock—it was a reminder.

> **The past doesn’t just tell us what has been—it warns us of what may come again.**

This project is not meant to be a conclusion, but a call to action:

* Historical data gives us context for current risks.
* It makes visible what society often forgets.
* And it empowers researchers, policymakers, and citizens alike to choose awareness over amnesia.

Because only if we are willing to learn from the past, can we be better prepared for the future—as a society, as a health system, and as individuals.
